<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/rooftops_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rooftops classification

In [5]:
import os
from pathlib import Path
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import layers
from skimage import io
from skimage import exposure
from skimage.io import imsave, imread
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import time
import warnings
warnings.filterwarnings('ignore')
print("NumPy version: %s"%np.__version__)
print('Tensorflow version: {:s}'.format(tf.__version__))
print('Keras version. {}'.format(keras.__version__))

NumPy version: 1.26.4
Tensorflow version: 2.17.1
Keras version. 3.5.0


In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
import zipfile
from zipfile import ZipFile
DATADIR = '.'

In [3]:
with ZipFile(f'{DATADIR}/alidoost_png_dataset.zip', 'r') as zipObj:
   zipObj.extractall(path=f'{DATADIR}/rooftops_dataset')

In [9]:
dataset_dir_path = Path('rooftops_dataset/png_dataset/train') # original images
subfolders_list = [os.path.basename(sub_folder) for sub_folder in dataset_dir_path.iterdir() if sub_folder.is_dir()]
subfolders_list

['hip', 'flat', 'gable']

In [10]:
png_dir = 'rooftops_dataset/png_dataset/'
png_dir_path = Path(png_dir)
png_dir_path

PosixPath('rooftops_dataset/png_dataset')

In [11]:
image_height = 224
image_width = 224
image_channels = 3

In [12]:
def build_cnn_model():
    inputs = keras.Input(shape=(image_height, image_width, image_channels))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu", name='1st_Conv_Layer')(inputs)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [13]:
rooftops_model = build_cnn_model()
rooftops_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ 1st_Conv_Layer (Conv2D)              │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 147456)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │         442,371 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 830,787 (3.17 MB)

 Trainable params: 830,787 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.utils import image_dataset_from_directory

batch_size = 32

train_dataset = image_dataset_from_directory(png_dir_path / 'train',
                                             labels='inferred',
                                             label_mode='int',
                                             image_size=(image_height, image_width),
                                             batch_size=batch_size)

validation_dataset = image_dataset_from_directory(png_dir_path / 'validation',
                                                  labels='inferred',
                                                  label_mode='int',
                                                  image_size=(image_height, image_width),
                                                  batch_size=batch_size)

test_dataset = image_dataset_from_directory(png_dir_path / 'test',
                                            labels='inferred',
                                            label_mode='int',
                                            image_size=(image_height, image_width),
                                            batch_size=batch_size)

Found 9010 files belonging to 3 classes.
Found 3000 files belonging to 3 classes.
Found 2412 files belonging to 3 classes.
